In [1]:
# ==========================================================
# 11_oneclass_gnn — Setup + Load (graphs & splits)
# ==========================================================

# --- Project root for imports/paths ---
import sys
from pathlib import Path

if "__file__" in globals():
    ROOT = Path(__file__).resolve().parents[1]
else:
    ROOT = Path.cwd().parents[0]  # notebook em /notebooks

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print(f"📁 ROOT: {ROOT.resolve()}")

# --- Imports ---
import torch
import numpy as np
import pandas as pd

from src.utils import ConfigLoader

# --- Config & paths ---
cfg = ConfigLoader.load("base.yaml")
data_proc = (ROOT / cfg["paths"]["data_processed"]).resolve()
splits_dir = data_proc / "splits"
graphs_path = data_proc / "elliptic_graphs.pt"

print("data_processed:", data_proc)
print("splits_dir    :", splits_dir)
print("graphs_path   :", graphs_path)

# --- Carrega splits tabulares (txId, time_step, class, features_*) para métricas/mascaras/mapas
df_train = pd.read_csv(splits_dir / "train_nodes.csv")
df_test1 = pd.read_csv(splits_dir / "test1_nodes.csv")
df_test2 = pd.read_csv(splits_dir / "test2_nodes.csv")

print("✅ splits CSV:", 
      "train", df_train.shape, 
      "test1", df_test1.shape, 
      "test2", df_test2.shape)

# --- Carregamento seguro dos grafos PyG salvos no 03 (compatível com PyTorch>=2.6)
from torch.serialization import safe_globals
from torch_geometric.data.data import Data as _PyGData
try:
    with safe_globals([_PyGData]):
        graphs = torch.load(graphs_path, weights_only=False, map_location="cpu")
except Exception as e:
    # Fallback (se necessário): permitir DataEdgeAttr antigos
    from torch_geometric.data.data import DataEdgeAttr as _PyGDataEdgeAttr  # type: ignore
    with safe_globals([_PyGData, _PyGDataEdgeAttr]):
        graphs = torch.load(graphs_path, weights_only=False, map_location="cpu")

assert isinstance(graphs, (list, tuple)) and len(graphs) > 0, "Falha ao carregar lista de grafos."
assert hasattr(graphs[0], "time_step") and hasattr(graphs[0], "y") and hasattr(graphs[0], "x")

# --- Resumo dos grafos e criação dos splits temporais ---
graphs_by_t = {int(g.time_step): g for g in graphs}
all_ts = sorted(graphs_by_t.keys())
in_channels = graphs[0].x.size(1)

# Intervalos (coerentes com notebooks anteriores)
train_range = (1, 34)
pre_range   = (35, 42)
post_range  = (43, 49)

train_graphs = [graphs_by_t[t] for t in range(train_range[0], train_range[1] + 1) if t in graphs_by_t]
test1_graphs = [graphs_by_t[t] for t in range(pre_range[0],   pre_range[1]   + 1) if t in graphs_by_t]
test2_graphs = [graphs_by_t[t] for t in range(post_range[0],  post_range[1]  + 1) if t in graphs_by_t]

print(f"✅ Loaded {len(graphs)} graphs | time_steps: {all_ts[:3]} ... {all_ts[-3:]}")
print(f"   in_channels = {in_channels}")
print(f"   splits => train:{len(train_graphs)}  pre:{len(test1_graphs)}  post:{len(test2_graphs)}")

# --- Sanity: prevalência média por split (usando rótulos != -1)
def _split_prevalence(graph_list):
    ys = []
    for g in graph_list:
        y = g.y.cpu().numpy()
        m = (y != -1)
        if m.any():
            ys.append(y[m])
    if not ys:
        return np.nan
    y_all = np.concatenate(ys)
    return float(np.mean(y_all))

print("   prev(train) ~", f"{_split_prevalence(train_graphs):.3f}")
print("   prev(pre)   ~", f"{_split_prevalence(test1_graphs):.3f}")
print("   prev(post)  ~", f"{_split_prevalence(test2_graphs):.3f}")

# --- Guarda alguns objetos úteis no workspace ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED_GLOBAL = cfg.get("general", {}).get("seed", 42)

print("DEVICE:", DEVICE)
print("SEED_GLOBAL:", SEED_GLOBAL)


📁 ROOT: /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC
data_processed: /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed
splits_dir    : /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed/splits
graphs_path   : /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed/elliptic_graphs.pt
✅ splits CSV: train (29894, 168) test1 (9983, 168) test2 (6687, 168)
✅ Loaded 49 graphs | time_steps: [1, 2, 3] ... [47, 48, 49]
   in_channels = 165
   splits => train:34  pre:8  post:7
   prev(train) ~ 0.116
   prev(pre)   ~ 0.092
   prev(post)  ~ 0.025
DEVICE: cpu
SEED_GLOBAL: 42


In [3]:
# ==========================================================
# Treino (5 seeds) de OCGCN / OCGAT / OCGraphSAGE + avaliação
# ==========================================================
import numpy as np
import pandas as pd
import torch
import inspect
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

from src.models.oc import OCGCN, OCGAT, OCGraphSAGE, OneClassTrainer

# ---------------------------
# Hiperparâmetros (puxa do YAML se houver; senão defaults)
# ---------------------------
oc_cfg = (
    cfg.get("models", {}).get("oneclass", {})
    or cfg.get("oc", {})
    or {}
)

def _hp_for(name: str):
    d = oc_cfg.get(name.lower(), {})
    return dict(
        hidden_channels = int(d.get("hidden_channels", 64)),
        epochs          = int(d.get("epochs", 200)),
        lr              = float(d.get("lr", 1e-3)),
        weight_decay    = float(d.get("weight_decay", 0.0)),  # opcional; trainer aceita
        dropout         = float(d.get("dropout", 0.5)),
    )

# ---------------------------
# Métricas auxiliares
# ---------------------------
def _calibrate_threshold_pre(df_pre: pd.DataFrame):
    m = (df_pre["y"] != -1)
    if not m.any():
        return None
    y = df_pre.loc[m, "y"].to_numpy()
    s = df_pre.loc[m, "score"].to_numpy()
    percentiles = np.linspace(5, 95, 91)
    best_f1, best_thr = -1.0, None
    for p in percentiles:
        thr = np.percentile(s, p)
        yhat = (s >= thr).astype(int)
        f1 = f1_score(y, yhat, pos_label=1, zero_division=0)
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr) if best_thr is not None else None

def _metrics_split(df: pd.DataFrame, thr: float):
    m = (df["y"] != -1)
    if not m.any():
        return {k: np.nan for k in ["Precision","Recall","F1","Gini","PR-AUC","Prevalência","PR-AUC/Prev"]}
    y = df.loc[m, "y"].to_numpy()
    s = df.loc[m, "score"].to_numpy()
    yhat = (s >= thr).astype(int) if thr is not None else np.zeros_like(y)
    precision = precision_score(y, yhat, pos_label=1, zero_division=0)
    recall    = recall_score(y, yhat, pos_label=1, zero_division=0)
    f1        = f1_score(y, yhat, pos_label=1, zero_division=0)
    try:
        roc_auc = roc_auc_score(y, s)
        gini    = 2*roc_auc - 1
    except ValueError:
        roc_auc = np.nan
        gini    = np.nan
    try:
        pr_auc  = average_precision_score(y, s)
    except ValueError:
        pr_auc  = np.nan
    prev = float(np.mean(y)) if len(y) else np.nan
    pr_over_prev = (pr_auc / prev) if (prev and prev > 0) else np.nan
    return {
        "Precision": precision,
        "Recall":    recall,
        "F1":        f1,
        "Gini":      gini,
        "PR-AUC":    pr_auc,
        "Prevalência": prev,
        "PR-AUC/Prev": pr_over_prev,
    }

def _metrics_by_timestep(df_all: pd.DataFrame, thr: float):
    out = []
    for ts, sub in df_all.groupby("time_step"):
        m = _metrics_split(sub, thr)
        out.append({
            "time_step": int(ts),
            "precision": m["Precision"],
            "recall":    m["Recall"],
            "f1":        m["F1"],
            "roc_auc":   (m["Gini"] + 1)/2 if not np.isnan(m["Gini"]) else np.nan,
            "pr_auc":    m["PR-AUC"],
            "gini":      m["Gini"],
            "prevalencia": m["Prevalência"],
            "pr_auc_over_prev": m["PR-AUC/Prev"],
        })
    return sorted(out, key=lambda d: d["time_step"])

# ---------------------------
# Adaptador de construtor: passa aliases conforme assinatura do modelo
# ---------------------------
def _instantiate_oneclass_model(model_ctor, in_channels, hp):
    sig = inspect.signature(model_ctor.__init__)
    valid = set(sig.parameters.keys())

    hidden_val = hp["hidden_channels"]
    hidden_aliases = {
        "hidden_channels": hidden_val,
        "hidden_dim":      hidden_val,
        "channels":        hidden_val,
        "out_channels":    hidden_val,
        "embedding_dim":   hidden_val,
        "latent_dim":      hidden_val,
        "dim":             hidden_val,
    }
    dropout_aliases = {
        "dropout": hp["dropout"],
        "p":       hp["dropout"],
    }

    kwargs = {}
    if "in_channels" in valid:
        kwargs["in_channels"] = in_channels
    for k, v in hidden_aliases.items():
        if k in valid:
            kwargs[k] = v
            break
    for k, v in dropout_aliases.items():
        if k in valid:
            kwargs[k] = v
            break

    return model_ctor(**kwargs).to(DEVICE)

# ---------------------------
# Coleta scores (sem usar trainer.embed/center_); usa center retornado por fit
# ---------------------------
@torch.no_grad()
def _collect_scores_with_center(model, center: torch.Tensor, graphs_list):
    rows = []
    model.eval().to(DEVICE)
    c = center.detach().to(DEVICE)
    for g in graphs_list:
        x = g.x.to(DEVICE)
        z = model(x, g.edge_index.to(DEVICE))  # (N, d)
        d = torch.sum((z - c)**2, dim=1).sqrt().detach().cpu().numpy()
        y = g.y.detach().cpu().numpy()
        ts = int(g.time_step)
        rows.append(pd.DataFrame({"time_step": ts, "y": y, "score": d}))
    return pd.concat(rows, ignore_index=True)

# ---------------------------
# Pipeline p/ 1 modelo/seed
# ---------------------------
def _run_one_model(model_name: str, model_ctor, seed: int):
    hp = _hp_for(model_name)  # hidden_channels, epochs, lr, weight_decay, dropout
    model = _instantiate_oneclass_model(model_ctor, in_channels, hp)

    trainer = OneClassTrainer(
        model=model,
        lr=hp["lr"],
        epochs=hp["epochs"],                # sua classe aceita 'epochs' (atalho min=max)
        weight_decay=hp["weight_decay"],    # opcional; se 0, não aplica
        device=str(DEVICE),
        seed=seed,
        val_period="pre",
    )
    # usa test1 como validação (pré) para early stopping/seleção
    center, _hist = trainer.fit(train_graphs, test1_graphs)

    # coleta scores nos splits de teste (pré/pós)
    df_pre  = _collect_scores_with_center(model, center, test1_graphs)
    df_post = _collect_scores_with_center(model, center, test2_graphs)
    df_glob = pd.concat([df_pre, df_post], ignore_index=True)

    # calibra threshold no pré (max F1)
    thr = _calibrate_threshold_pre(df_pre)

    # métricas
    m_pre  = _metrics_split(df_pre,  thr)
    m_post = _metrics_split(df_post, thr)
    m_glob = _metrics_split(df_glob, thr)
    by_ts  = _metrics_by_timestep(df_glob, thr)

    return {
        "Global (35–42)": m_pre,
        "Global (43–49)": m_post,
        "Global (35–49)": m_glob,
        "por_time_step": by_ts,
        "seed": seed,
    }

# ---------------------------
# Execução (5 seeds) para cada modelo
# ---------------------------
seeds = [SEED_GLOBAL + i for i in range(5)]
results_summary_oc = {"OCGCN": [], "OCGAT": [], "OCGraphSAGE": []}

for seed in seeds:
    print(f"\n=== Seed {seed} ===")
    print("-> OCGCN")
    results_summary_oc["OCGCN"].append(_run_one_model("ocgcn", OCGCN, seed))
    print("-> OCGAT")
    results_summary_oc["OCGAT"].append(_run_one_model("ocgat", OCGAT, seed))
    print("-> OCGraphSAGE")
    results_summary_oc["OCGraphSAGE"].append(_run_one_model("ocgraphsage", OCGraphSAGE, seed))

print("\n✅ Concluído. Modelos avaliados:", list(results_summary_oc.keys()))
for name, runs in results_summary_oc.items():
    mean_pre = np.nanmean([r["Global (35–42)"]["PR-AUC"] for r in runs])
    print(f" {name}: PR-AUC(pre) média = {mean_pre:.4f}")



=== Seed 42 ===
-> OCGCN
-> OCGAT
-> OCGraphSAGE

=== Seed 43 ===
-> OCGCN
-> OCGAT
-> OCGraphSAGE

=== Seed 44 ===
-> OCGCN
-> OCGAT
-> OCGraphSAGE

=== Seed 45 ===
-> OCGCN
-> OCGAT
-> OCGraphSAGE

=== Seed 46 ===
-> OCGCN
-> OCGAT
-> OCGraphSAGE

✅ Concluído. Modelos avaliados: ['OCGCN', 'OCGAT', 'OCGraphSAGE']
 OCGCN: PR-AUC(pre) média = 0.0804
 OCGAT: PR-AUC(pre) média = nan
 OCGraphSAGE: PR-AUC(pre) média = 0.0872


/var/folders/0j/y0hjl4v54fld89f_fkf4j7x40000gn/T/ipykernel_56322/3271557965.py:213: RuntimeWarning: Mean of empty slice
  mean_pre = np.nanmean([r["Global (35–42)"]["PR-AUC"] for r in runs])


In [4]:
# ==========================================================
# Resumo (tabela) — OCGCN vs OCGAT vs OCGraphSAGE
# ==========================================================
import numpy as np
import pandas as pd
from IPython.display import display

# results_summary_oc deve conter as chaves:
# "OCGCN", "OCGAT", "OCGraphSAGE"
print("Chaves encontradas:", list(results_summary_oc.keys()))

results_summary = {
    "OC": results_summary_oc
}

def mean_std_str(values):
    arr = np.array(values, dtype=float)
    return f"{np.nanmean(arr):.3f} ± {np.nanstd(arr):.3f}"

summary_rows = []
for family_key, model_dict in results_summary.items():
    for model_name, runs in model_dict.items():
        for label, key in [
            ("Pré (35–42)", "Global (35–42)"),
            ("Pós (43–49)", "Global (43–49)"),
            ("Global (35–49)", "Global (35–49)"),
        ]:
            summary_rows.append({
                "Família": family_key,
                "Modelo": model_name,
                "Período": label,
                "Gini":          mean_std_str([r[key]["Gini"] for r in runs]),
                "PR-AUC":        mean_std_str([r[key]["PR-AUC"] for r in runs]),
                "PR-AUC/Prev":   mean_std_str([r[key]["PR-AUC/Prev"] for r in runs]),
                "F1":            mean_std_str([r[key]["F1"] for r in runs]),
                "Recall":        mean_std_str([r[key]["Recall"] for r in runs]),
                "Precision":     mean_std_str([r[key]["Precision"] for r in runs]),
            })

df_summary_oc = (
    pd.DataFrame(summary_rows)
      .set_index(["Família","Modelo","Período"])
      .sort_index()
)

# Opcional: reordenar linhas por Modelo em ordem específica
order = ["OCGCN", "OCGAT", "OCGraphSAGE"]
df_summary_oc = df_summary_oc.reindex(
    pd.MultiIndex.from_product(
        [["OC"], order, ["Pré (35–42)", "Pós (43–49)", "Global (35–49)"]],
        names=["Família","Modelo","Período"]
    )
)

display(df_summary_oc)


Chaves encontradas: ['OCGCN', 'OCGAT', 'OCGraphSAGE']


/var/folders/0j/y0hjl4v54fld89f_fkf4j7x40000gn/T/ipykernel_56322/2919678842.py:18: RuntimeWarning: Mean of empty slice
  return f"{np.nanmean(arr):.3f} ± {np.nanstd(arr):.3f}"
/Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/.venv_gnn/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Gini         PR-AUC  \
Família Modelo      Período                                         
OC      OCGCN       Pré (35–42)     -0.055 ± 0.093  0.080 ± 0.007   
                    Pós (43–49)     -0.057 ± 0.049  0.024 ± 0.001   
                    Global (35–49)  -0.092 ± 0.071  0.055 ± 0.003   
        OCGAT       Pré (35–42)          nan ± nan      nan ± nan   
                    Pós (43–49)          nan ± nan      nan ± nan   
                    Global (35–49)       nan ± nan      nan ± nan   
        OCGraphSAGE Pré (35–42)     -0.044 ± 0.089  0.087 ± 0.004   
                    Pós (43–49)     -0.074 ± 0.133  0.024 ± 0.003   
                    Global (35–49)  -0.049 ± 0.064  0.061 ± 0.003   

                                      PR-AUC/Prev             F1  \
Família Modelo      Período                                        
OC      OCGCN       Pré (35–42)     0.878 ± 0.079  0.182 ± 0.012   
                    Pós (43–49)     0.951 ± 0.057  0.050 ± 0.004   
                    Global (35–49)  0.844 ± 0.054  0.129 ± 0.006   
        OCGAT       Pré (35–42)         nan ± nan  0.000 ± 0.000   
                    Pós (43–49)         nan ± nan  0.000 ± 0.000   
                    Global (35–49)      nan ± nan  0.000 ± 0.000   
        OCGraphSAGE Pré (35–42)     0.953 ± 0.047  0.177 ± 0.008   
                    Pós (43–49)     0.939 ± 0.101  0.046 ± 0.009   
                    Global (35–49)  0.939 ± 0.043  0.127 ± 0.003   

                                           Recall      Precision  
Família Modelo      Período                                       
OC      OCGCN       Pré (35–42)     0.876 ± 0.080  0.102 ± 0.009  
                    Pós (43–49)     0.878 ± 0.143  0.026 ± 0.002  
                    Global (35–49)  0.876 ± 0.090  0.070 ± 0.004  
        OCGAT       Pré (35–42)     0.000 ± 0.000  0.000 ± 0.000  
                    Pós (43–49)     0.000 ± 0.000  0.000 ± 0.000  
                    Global (35–49)  0.000 ± 0.000  0.000 ± 0.000  
        OCGraphSAGE Pré (35–42)     0.909 ± 0.084  0.098 ± 0.006  
                    Pós (43–49)     0.819 ± 0.235  0.024 ± 0.004  
                    Global (35–49)  0.895 ± 0.108  0.068 ± 0.002

In [5]:
for name, glist in [("pre", test1_graphs), ("post", test2_graphs)]:
    ys = np.concatenate([g.y[g.y!=-1].cpu().numpy() for g in glist if (g.y!=-1).any()])
    print(name, "len=", len(ys), "positives=", (ys==1).sum(), "negatives=", (ys==0).sum())


pre len= 9983 positives= 914 negatives= 9069
post len= 6687 positives= 169 negatives= 6518


In [6]:
with torch.no_grad():
    g = test1_graphs[0].to(DEVICE)
    z = trained_ocgat.embed(g)  # usa o trainer e o modelo já treinados
    print("has_nan:", torch.isnan(z).any().item(),
          "has_inf:", torch.isinf(z).any().item(),
          "std:", z.std().item())


NameError: name 'trained_ocgat' is not defined